# Machine Learning Lifecycle

Machine learning projects are highly iterative; as you progress through the ML lifecycle, you’ll find yourself iterating on a section until reaching a satisfactory level of performance, then proceeding forward to the next task (which may be circling back to an even earlier step)

[<img src="img/lab_05_cycle.png"/>](https://www.jeremyjordan.me/ml-projects-guide/)


Often times, we don't immediately know what the optimal model architecture should be for a given model, and thus we'd like to be able to explore a range of possibilities. **Start simple and gradually ramp up complexity**. This typically involves using a simple model, but can also include starting with a simpler version of your task, and data preparation.

Once you have a general idea of successful model architectures and approaches for your problem, you should now spend much more focused effort on squeezing out performance gains including from the model. That's why we use **pipelines**.




# Preprocessing & Feature Engineering

Preprocessing and feature engineering efforts mainly have two goals:

1. Preparing the proper input dataset, compatible with the machine learning algorithm requirements.
2. Improving the performance of machine learning models.


There are different techniques and steps for preprocessing and feature engineering based on the dataset and/or algorithms.

Here is a list of major feature engineering techniques.

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other.

For example, **age** and **income** don't have the same range in real life, but from the machine learning point of view, we need a way to compare these two columns.

Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but in general, ML models benefit from scaling of the data.

Remeber, if your algorithm is based on **distance** calculations such as k-NN or k-Means need to have scaled continuous features as model input.

* **Normalization**:  is the process of scaling individual samples to have unit norm.

X_new = (X - X_min)/(X_max - X_min)

* **Standardization**: Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with **zero mean and unit variance**.
X_new = (X - mean)/Std

In [ ]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
scaler.fit(data)
print(f"Original data: {data}")

print(f"Mean: {scaler.mean_}")

scaler.transform(data)


# print(scaler.transform([[2, 2]]))

----
## Imputation

Missing values are one of the most common problems you can encounter when you try to prepare your data for machine learning. The reason for the missing values might be human errors, interruptions in the data flow, privacy concerns, and so on. Whatever is the reason, missing values affect the performance of the machine learning models.

**Dropping missing values**: The most simple solution to the missing values is to drop the rows or the entire column.

**Numerical imputation**: using Mean/Median values. This works by calculating the mean/median of the non-missing **values in** a column and then replacing the missing values within each column separately and independently from the others. It can only be used with numeric data.


**Categorical imputation**: Replacing the missing values with the maximum occurred value in a column is a good option for handling categorical columns.

The above methods don’t factor the correlations between features. It only works on the column level.

**ML-based imputation** for example, **K-NN** uses ‘feature similarity’ to predict the values of any new data points. This means that the new point is assigned a value based on how closely it resembles the points in the training set. This can be very useful in making predictions about the missing values by finding the k’s closest neighbours to the observation with missing data and then imputing them based on the non-missing values in the neighbourhood

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])

X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))

----
## Handling outliers

Many applications require being able to decide whether a new observation belongs to the same distribution as existing observations (it is an inlier), or should be considered as different (it is an outlier). Often, this ability is used to clean real data sets.. Outliers in data can have many causes such as 1-data entry error (human error), 2-Measurement errors (instrument errors), 3- Experimental errors, 4- Data processing errors, 4- Sampling errors, 5. Natural (not an error, novelties in data) and so on.

Usually the best way to detect the outliers is to demonstrate the data visually (Histograms, Scatter plot, box plots, ...). However, if visualization is not possible, there are statistical methods to detect outliers such as:
* Z-Score or Extreme Value Analysis (parametric): The z-score or standard score of an observation is a metric that indicates how many standard deviations a data point is from the sample’s mean, assuming a gaussian distribution.


Often outliers are discarded because of their effect on the total distribution and statistical analysis of the dataset. This is certainly a good approach if the outliers are due to an error of some kind (measurement error, data corruption, etc.), however often the source of the outliers is unclear. There are many situations where occasional ‘extreme’ events cause an outlier that is outside the usual distribution of the dataset but is a valid measurement and not due to an error. In these situations, the choice of how to deal with the outliers is not necessarily clear and the choice has a significant impact on the results of any statistical analysis done on the dataset. The decision about how to deal with outliers depends on the goals and context of the research and should be detailed in any explanation about the methodology.


There are some techniques used to deal with outliers.
* Deleting observations
* Transforming values: Use **robust** transformers
* Separately treating

In [ ]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd
import seaborn as sns

#Load data
X, y = load_boston(return_X_y=True)

#Create data frame
boston = load_boston()
columns = boston.feature_names
df = pd.DataFrame(X, columns = columns)

ax = sns.boxplot(data=df[['CRIM', 'ZN']], orient="h", palette="Set2")

----
## Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized.


Binning can be applied on both categorical and numerical data.

In [ ]:
df['CRIM'].describe()

The KBinsDiscretizer can then be used to convert the floating values into fixed number of discrete categories with an ranked ordinal relationship

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer  

est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
df['CRIM_binned'] = est.fit_transform(df[['CRIM']])

In [ ]:
df['CRIM_binned'].describe()

Another way is to use `Pandas qcut`. The function defines the bins using percentiles based on the distribution of the data, not the actual numeric edges of the bins.

In [ ]:
df['CRIM_binned_pd'] = pd.qcut(df['CRIM'], 3, labels=False)

In [ ]:
df['CRIM_binned_pd'].describe()

-----
## Encoding categorical features

Often features are not given as continuous values but categorical.

**OrdinalEncoder** is used to convert categorical features to such integer codes. This estimator transforms each categorical feature to one new feature of integers (0 to n_categories - 1)


**One-hot encoding** is one of the most common encoding methods in machine learning. This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format and enables you to group your categorical data without losing any information. (For details please see the last part of Categorical Column Grouping)

<img src="img/lab_05_onehot.png"/>

----

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from numpy import asarray

# define data
data = asarray([['good'], ['better'], ['best']])
print(data)
# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
result = encoder.fit_transform(data)
print(result)

In [ ]:
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
# define data
data = asarray([['red'], ['green'], ['blue']])
print(data)
# define one hot encoding
encoder = OneHotEncoder(sparse=False)
# transform data
onehot = encoder.fit_transform(data)
print(onehot)

The problem is that this representation includes redundancy. For example, if we know that [1, 0, 0] represents “blue” and [0, 1, 0] represents “green” we don’t need another binary variable to represent “red“, instead we could use 0 values for both “blue” and “green” alone, e.g. [0, 0].

This is called a **dummy variable encoding**, and always represents C categories with C-1 binary variables.

In [ ]:
# example of a dummy variable encoding
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
# define data
data = asarray([['red'], ['green'], ['blue']])
print(data)
# define one hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)
# transform data
onehot = encoder.fit_transform(data)
print(onehot)

One-hot encoding/getting dummies Pandas way...

In [ ]:
df_data = pd.DataFrame(data)
print(df_data)

In [ ]:
pd.get_dummies(df_data,  drop_first=True)

##  <font color='red'>Interatction/new features </font>



----
# Feature Selection

Feature selection is the process of reducing the number of input variables when developing a predictive model.

It is desirable to reduce the number of input variables to both reduce the computational cost of modeling and, in some cases, to improve the performance of the model.

https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/

# Hyperparameter Tuning

A model **hyperparameter** is a configuration that is external to the model and whose value cannot be estimated from data.

* They are often used in processes to help estimate model parameters.
* They are often specified by the practitioner.
* They can often be set using heuristics.
* They are often tuned for a given predictive modeling problem.

Hyperparameters address model design questions such as:
* K in K-NN
* What should be the maximum depth allowed for my decision tree?
* How many trees should I include in my random forest?
* How many layers should I have in my neural network?


We cannot know the best value for a model hyperparameter on a given problem. We may use rules of thumb, copy values used on other problems, or search for the best value by trial and error.

[<img src="img/lab_05_hpt.png"/>](https://www.analyticsvidhya.com/blog/2021/04/evaluating-machine-learning-models-hyperparameter-tuning/)

-----
Searching for the best hyper-parameter can be tedious, hence search algorithms like **grid search** or **random search**. USing these, you are tuning the hyperparameters of the model in order to discover the parameters of the model that result in the most skillful predictions.

Use **coarse-to-fine random searches** for hyperparameters. Start with a wide hyperparameter space initially and iteratively hone in on the highest-performing region of the hyperparameter space.

Please note that optimal hyperparameters often differ for different datasets and models.

A search consists of:

* an estimator (regressor or classifier such as sklearn.svm.SVC());
* a parameter space;
* a method for searching or sampling candidates;
* a cross-validation scheme; and
* a score function.


In [9]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split


data = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, stratify=data.target, random_state=0)

In [10]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.

# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=100, tol=0.1)
pipe = Pipeline(steps=[('scaler', StandardScaler()),
                       ('logreg', logistic)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'logreg__C': np.logspace(-4, 4, 4),
    'logreg__penalty':['l2', 'l1'],
    'logreg__class_weight':['balanced', 'none']
    
}
search = GridSearchCV(pipe, param_grid,
                      scoring='f1_macro',
                      cv=5,
                      n_jobs=-1)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.982):
{'logreg__C': 0.046415888336127774, 'logreg__class_weight': 'balanced', 'logreg__penalty': 'l2'}


/Users/imaniai/opt/anaconda3/envs/pratt/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imaniai/opt/anaconda3/envs/pratt/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imaniai/opt/anaconda3/envs/pratt/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imaniai/opt/anaconda3/envs/pratt/lib/python3.8/site-packages/sklearn/

In [11]:
y_pred = search.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        53
           1       0.96      0.97      0.96        90

    accuracy                           0.95       143
   macro avg       0.95      0.95      0.95       143
weighted avg       0.95      0.95      0.95       143



In [12]:
search.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegression(C=0.046415888336127774,
                                    class_weight='balanced', tol=0.1))])

In [29]:
new_lr = search.best_estimator_['logreg']
new_lr.fit(X_train, y_train)

y_pred = new_lr.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91        53
           1       0.95      0.93      0.94        90

    accuracy                           0.93       143
   macro avg       0.92      0.93      0.93       143
weighted avg       0.93      0.93      0.93       143



/Users/imaniai/opt/anaconda3/envs/pratt/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


----
So far, we built machine learning models and trained it on some data... now what?

There are still questions that need to be answered like:

* How well is my model doing? Is it a useful model?
* Will training my model on more data improve its performance?
* Do I need to include more features?

# Model Evaluation

Establish performance baselines on your problem. Baselines are useful for both establishing a lower bound of expected performance (simple model baseline) and establishing a target performance level (human baseline).

* Simple baselines include out-of-the-box scikit-learn models (i.e. logistic regression with default parameters) or even simple heuristics (always predict the majority class). Without these baselines, it's impossible to evaluate the value of added model complexity.
* If your problem is well-studied, search the literature to approximate a baseline based on published results for very similar tasks/datasets.
* If possible, try to estimate human-level performance on the given task. Don't naively assume that humans will perform the task perfectly, a lot of simple tasks are deceptively hard!

## Why is my model performing poorly?

There a many reasons why your model is not performing as you like it.

* Implementation bugs
* Hyperparameter choices
* Data/model fit
* Dataset construction


### Train/Test split

First step to properly evaluate your model is to not train the model on the entire dataset. I repeat: **do not train the model on the entire dataset.**


<img src="img/lab_05_split.png"/>

The training set contains a known output and the model learns on this data in order to be generalized to other data later on. We have the test dataset (or subset) in order to test our model’s prediction on this subset.



### Metrics

There are a variety of metrics you can use to evaluate your regression, clssification, and clustering models. You can read about many of them [here](https://scikit-learn.org/stable/modules/model_evaluation.html)

### Regression metrics

### Classification metrics
* imbalanced data

### Clustering metrics

## Precision Recall

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


data = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, stratify=data.target, random_state=0)

lr = LogisticRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
data.data.shape

In [ ]:
from sklearn.metrics import confusion_matrix

# print(confusion_matrix(y_test, y_pred))
print(lr.score(X_test, y_test))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# Precision Recall Curve

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
import numpy as np

iris = datasets.load_iris()
X = iris.data
y = iris.target

# Add noisy features
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# Limit to the two first classes, and split into training and test
X_train, X_test, y_train, y_test = train_test_split(X[y < 2], y[y < 2],
                                                    test_size=.5,
                                                    random_state=random_state)

# Create a simple classifier
classifier = svm.LinearSVC(random_state=random_state)
classifier.fit(X_train, y_train)
y_score = classifier.decision_function(X_test)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(classifier, X_test, y_test)

# ROC Curve

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot


# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(testy))]
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

# Imbalanced data

In [ ]:
from sklearn.datasets import fetch_openml
# mammography dataset https://www.openml.org/d/310
data = fetch_openml('mammography')
X, y = data.data, data.target
y = (y.astype(np.int) + 1) // 2

In [ ]:
np.bincount(y)

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)


lr = LogisticRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)

print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
lr = LogisticRegression(class_weight='balanced').fit(X_train, y_train)
y_pred = lr.predict(X_test)

print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#!pip install --user imblearn

In [ ]:
X_train.shape

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(replacement=False)
X_train_subsample, y_train_subsample = rus.fit_sample(X_train, y_train)

print(X_train.shape)
print(X_train_subsample.shape)
print(np.bincount(y_train_subsample))


In [ ]:
lr = LogisticRegression().fit(X_train_subsample, y_train_subsample)
y_pred = lr.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.over_sampling import RandomOverSampler

oversample_pipe = make_imb_pipeline(RandomOverSampler(),
                                    LogisticRegression())

oversample_pipe.fit(X_train, y_train)
y_pred = oversample_pipe.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
X_train_oversample, y_train_oversample = RandomOverSampler().fit_sample(X_train, y_train)


print(X_train.shape)
print(X_train_oversample.shape)
print(np.bincount(y_train_oversample))

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve

probs_oversample = oversample_pipe.predict_proba(X_test)[:, 1]
fpr_over, tpr_over, _ = roc_curve(y_test, probs_oversample)
precision_over, recall_over, _ = precision_recall_curve(y_test, probs_oversample)

undersample_pipe = make_imb_pipeline(RandomUnderSampler(),
                                    LogisticRegression())
undersample_pipe.fit(X_train, y_train)
probs_undersample = undersample_pipe.predict_proba(X_test)[:, 1]
fpr_under, tpr_under, _ = roc_curve(y_test, probs_undersample)
precision_under, recall_under, _ = precision_recall_curve(y_test, probs_undersample)

lr = LogisticRegression().fit(X_train, y_train)
probs_original = lr.predict_proba(X_test)[:, 1]
fpr_org, tpr_org, _ = roc_curve(y_test, probs_original)
precision_org, recall_org, _ = precision_recall_curve(y_test, probs_original)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].plot(fpr_org, tpr_org, label="original", alpha=.9)
axes[0].plot(fpr_over, tpr_over, label="oversample", alpha=.9)
axes[0].plot(fpr_under, tpr_under, label="undersample", alpha=.9)
axes[0].legend()
axes[0].set_xlabel("FPR")
axes[0].set_ylabel("TPR")
axes[0].set_title("LogReg ROC curve")

axes[1].plot(recall_org, precision_org, label="original", alpha=.9)
axes[1].plot(recall_over, precision_over, label="oversample", alpha=.9)
axes[1].plot(recall_under, precision_under, label="undersample", alpha=.9)
axes[1].legend()
axes[1].set_xlabel("recall")
axes[1].set_ylabel("precision")
axes[1].set_title("LogReg PR curve")
plt.tight_layout()

# Explainability

In [ ]:
#!pip install --user shap
# !pip install --user numba

In [ ]:
new_lr.coef_

In [ ]:
import pandas as pd

plt.figure(figsize=(10, 4))
df = pd.DataFrame(new_lr.coef_)
ax = plt.gca()
df.plot.bar(ax=ax, width=.9)
ax.set_ylim(-1.5, 1.5)
ax.set_xlim(-.5, len(df) - .5)
ax.set_xlabel("feature index")
ax.set_ylabel("importance value")
plt.vlines(np.arange(.5, len(df) -1), -1.5, 1.5, linewidth=.5)

In [ ]:
import shap

explainer = shap.LinearExplainer(new_lr, X_train, feature_dependence="independent")
shap_values = explainer.shap_values(X_test)


## Summarize the effect of all the features


Shapley values calculate the importance of a feature by comparing what a model predicts with and without the feature. However, since the order in which a model sees features can affect its predictions, this is done in every possible order, so that the features are fairly compared.

* The y-axis indicates the variable name, in order of importance from top to bottom. The value next to them is the mean SHAP value.
* On the x-axis is the SHAP value. Indicates how much is the change in log-odds. From this number we can extract the probability of success.
* Gradient color indicates the original value for that variable. In booleans, it will take two colors, but in number it can contain the whole spectrum.
* Each point represents a row from the original dataset.

In [ ]:
shap.summary_plot(shap_values, X_test)


In [ ]:
# try this with randomforests